# Step 4: V01 — Odd/Even depth comparison

Goal: check whether odd and even transits have consistent depths.

Why this matters:
- A significant odd/even mismatch can indicate an eclipsing binary at twice/half the true period.
- Even when the odd/even depth statistic looks fine, it’s still valuable to look at the odd/even folded light curve.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V01')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "delta_ppm": -68.6,
    "delta_sigma": 1.73,
    "depth_diff_sigma": 1.73,
    "depth_err_even_ppm": 30.6,
    "depth_err_odd_ppm": 25.2,
    "depth_even_ppm": 297.6,
    "depth_odd_ppm": 229.0,
    "even_depth": 0.000298,
    "method": "per_epoch_median",
    "n_even_points": 487,
    "n_even_transits": 4,
    "n_odd_points": 485,
    "n_odd_transits": 4,
    "odd_depth": 0.000229,
    "rel_diff": 0.23
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V01: Odd/Even depth comparison
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_odd_even
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='04_v01_odd_even')
    run_path = run_out_dir / 'V01_odd_even.png'
    docs_path = (docs_out_dir / 'V01_odd_even.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_odd_even(r, ax=ax)
    ax.set_title('V01: Odd/Even depth comparison')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even.png`

![V01: Odd/Even depth comparison](../artifacts/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even.png",
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V01: Odd/Even phase-folded transit (stay close to the data)
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import numpy as np
    import matplotlib.pyplot as plt
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE:
    t = np.asarray(lc.time, dtype=np.float64)
    f = np.asarray(lc.flux, dtype=np.float64)

    period = float(candidate.ephemeris.period_days)
    t0 = float(candidate.ephemeris.t0_btjd)

    # Fold into [-0.5, 0.5) phase, with phase=0 at mid-transit.
    phase = ((t - t0) / period + 0.5) % 1.0 - 0.5

    # Assign each point to the nearest transit epoch.
    epoch = np.floor((t - t0) / period + 0.5).astype(int)
    is_odd = (epoch % 2) != 0

    # Focus on a transit-centric window.
    phase_min, phase_max = -0.15, 0.15
    w = (phase >= phase_min) & (phase <= phase_max) & np.isfinite(f)

    x_hours = phase[w] * period * 24.0
    y_ppm = (f[w] - np.nanmedian(f[w])) * 1e6

    # Split odd vs even.
    x_odd = x_hours[is_odd[w]]
    y_odd = y_ppm[is_odd[w]]
    x_even = x_hours[~is_odd[w]]
    y_even = y_ppm[~is_odd[w]]

    # Simple binning for readability.
    def _binned(x: np.ndarray, y: np.ndarray, *, nbins: int = 60) -> tuple[np.ndarray, np.ndarray]:
        if len(x) == 0:
            return np.asarray([]), np.asarray([])
        edges = np.linspace(phase_min * period * 24.0, phase_max * period * 24.0, nbins + 1)
        idx = np.digitize(x, edges) - 1
        xc, yc = [], []
        for i in range(nbins):
            m = idx == i
            if m.sum() < 10:
                continue
            xc.append(float(np.nanmedian(x[m])))
            yc.append(float(np.nanmedian(y[m])))
        return np.asarray(xc), np.asarray(yc)

    bx_odd, by_odd = _binned(x_odd, y_odd)
    bx_even, by_even = _binned(x_even, y_even)

    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='04_v01_odd_even')
    run_path = run_out_dir / 'V01_odd_even_folded.png'
    docs_path = (docs_out_dir / 'V01_odd_even_folded.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    ax.scatter(x_odd, y_odd, s=6, alpha=0.35, color='tab:red', label='Odd (points)')
    ax.scatter(x_even, y_even, s=6, alpha=0.35, color='tab:green', label='Even (points)')
    if len(bx_odd):
        ax.plot(bx_odd, by_odd, color='tab:red', lw=2, label='Odd (binned)')
    if len(bx_even):
        ax.plot(bx_even, by_even, color='tab:green', lw=2, label='Even (binned)')

    ax.axvline(0.0, color='k', lw=1, alpha=0.25)
    ax.set_xlabel('Hours from mid-transit')
    ax.set_ylabel('Relative flux (ppm)')
    ax.set_title('V01: Odd/Even phase-folded transit (stitched PDCSAP)')
    ax.legend(loc='best')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even_folded.png`

![V01: Odd/Even phase-folded transit](../artifacts/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even_folded.png)


<details>
<summary><b>Expected Output (folded plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even_folded.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/04_v01_odd_even/V01_odd_even_folded.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** odd vs even depths differ by **1.73σ** (odd≈229.0±25.2 ppm, even≈297.6±30.6 ppm).
- **Why it’s useful:** a large odd/even mismatch is a classic EB red flag (wrong period); we don’t see that here.
- **Interpretation:** mild tension only; consistent with noise + small-number statistics.
- **Next step:** V02 (secondary eclipse at phase ∼0.5).

</details>
